In [0]:
# ======================================================
# Eploratory Data Analysis ON GOLD LAYER - Databricks Notebook (PySpark)
# ======================================================

from pyspark.sql import functions as F
from pyspark.sql import types as T

gold_table = "abc.abc_dw_gold.abc_dw_gl_pr_po_kpi"

# ------------------------------------------------------
# 1. LOAD GOLD DATASET
# ------------------------------------------------------
df = spark.table(gold_table)
display(df)

print("Record Count:", df.count())
print("Column Count:", len(df.columns))

# ------------------------------------------------------
# 2. SCHEMA REVIEW
# ------------------------------------------------------
df.printSchema()

# ------------------------------------------------------
# 3. SUMMARY STATISTICS
# ------------------------------------------------------
numeric_cols = [
    "pr_approval_ageing",
    "po_approval_ageing",
    "pr_to_po_ageing"
]

df.select(numeric_cols).describe().show()

# ------------------------------------------------------
# 4. MISSING VALUE CHECK
# ------------------------------------------------------
missing_df = df.select([
    F.sum(F.col(c).isNull().cast("int")).alias(c)
    for c in df.columns
])

display(missing_df)

# OPTIONAL — PERCENTAGE MISSING
missing_pct = df.select([
    (F.sum(F.col(c).isNull().cast("int")) / df.count() * 100).alias(c)
    for c in df.columns
])
display(missing_pct)

# ------------------------------------------------------
# 5. DISTRIBUTION OF KEY KPIs
# ------------------------------------------------------
for col in numeric_cols:
    print(f"Distribution for: {col}")
    display(df.select(col))

# (Use Databricks visualization → Histogram)

# ------------------------------------------------------
# 6. CORRELATION ANALYSIS (NUMERIC KPIs)
# ------------------------------------------------------
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
df_vec = assembler.transform(df)
corr_matrix = Correlation.corr(df_vec, "features")

#corr_matrix.show(truncate=False)
assembler = VectorAssembler(
    inputCols=numeric_cols,
    outputCol="features",
    handleInvalid="skip"
)
df_vec = assembler.transform(df)
corr_matrix = Correlation.corr(df_vec, "features")
display(corr_matrix)

# ------------------------------------------------------
# 7. PR & PO TREND ANALYSIS
# ------------------------------------------------------
# Convert date strings to date (if needed)
df = df.withColumn("pr_creationdate", F.to_date("pr_creationdate")) \
       .withColumn("po_createdon", F.to_date("po_createdon"))

# PR Volume Trend
display(
    df.groupBy("pr_creationdate").count().orderBy("pr_creationdate")
)

# PO Volume Trend
display(
    df.groupBy("po_createdon").count().orderBy("po_createdon")
)

# ------------------------------------------------------
# 8. VENDOR PERFORMANCE ANALYSIS
# ------------------------------------------------------
if "po_vendornumber" in df.columns:
    vendor_perf = (
        df.groupBy("po_vendornumber")
        .agg(F.avg("pr_to_po_ageing").alias("avg_pr_to_po_ageing"))
        .orderBy("avg_pr_to_po_ageing")
    )
    display(vendor_perf)

# ------------------------------------------------------
# 9. MATERIAL GROUP PERFORMANCE
# ------------------------------------------------------
if "materialgroupdesc" in df.columns:
    material_perf = (
        df.groupBy("materialgroupdesc")
        .agg(F.avg("pr_to_po_ageing").alias("avg_cycle"))
        .orderBy(F.desc("avg_cycle"))
    )
    display(material_perf)

# ------------------------------------------------------
# 10. SLA BREACH ANALYSIS
# ------------------------------------------------------
sla_summary = (
    df.groupBy("sla_breach_flag")
    .count()
    .orderBy("sla_breach_flag")
)
display(sla_summary)

# PR SLA Breach
display(
    df.groupBy("pr_cycle_sla_breach_flag")
    .count()
)

# PO SLA Breach
display(
    df.groupBy("po_cycle_sla_breach_flag")
    .count()
)

# ------------------------------------------------------
# 11. RECORD TYPE BREAKDOWN
# ------------------------------------------------------
record_type_counts = df.groupBy("record_type").count()
display(record_type_counts)

# ------------------------------------------------------
# 12. OUTLIER DETECTION
# ------------------------------------------------------
for col in numeric_cols:
    print(f"Checking outliers for: {col}")
    display(df.select(col))

# (Use Boxplot visualization in Databricks UI)

# ------------------------------------------------------
# 13. BIAS / ETHICS ANALYSIS
# ------------------------------------------------------
# Check ageing by department / company code
if "pr_companycode" in df.columns:
    display(
        df.groupBy("pr_companycode")
        .agg(F.avg("pr_to_po_ageing").alias("avg_delay"))
        .orderBy(F.desc("avg_delay"))
    )

# Check vendor country bias
if "po_countrykey" in df.columns:
    display(
        df.groupBy("po_countrykey")
        .agg(F.avg("pr_to_po_ageing").alias("avg_delay"))
        .orderBy(F.desc("avg_delay"))
    )

print("EDA Complete.")


pr_purchaserequisition,pr_itemnumber,pr_documenttype,pr_creationdate,pr_createdby_name,pr_approvalstatus,pr_approveddate,pr_processingstatus,pr_companycode,pr_companycodedesc,pr_plant,pr_plantdesc,pr_orderqty,pr_unitofmeasure,pr_closedindicator,pr_deletionindicator,pr_recordskipindicator,pr_lastchangedatetime,pr_silver_load_date,pr_silver_load_timestamp,purchaseorder,po_itemnumber,po_purchaserequisition,po_serialnumber,po_returnsitem,po_purchasingdoctype,po_purchasingdoctypedesc,po_purchasingdocdate,po_deletionindicator,po_purchasingorg,po_purchasingorgdesc,po_companycode,po_companycodedesc,po_purchasinggroup,po_purchasinggroupdesc,po_vendornumber,po_vendorname,po_vendoraccountgroup,po_vendoraccountgroupdesc,po_supplierclassification,po_countrykey,po_localorinternational,po_plant,po_plantdesc,po_storageloc,po_createdon,po_createdby_name,po_documentyear,po_accountassignmentcat,po_accountassignmentcatdesc,po_accntassignmentcatvaluedesc,po_version,po_processingstatuscode,po_processingstatus,po_postingdate,po_approvaldate,po_deliverydate,po_paymentdate,material,materialdesc,materialgroup,materialgroupdesc,materialtype,materialtypedesc,po_orderquantity,po_unitofmeasure,po_netamount,po_recordskipindicator,po_lastchangedatetime,po_silver_load_date,po_silver_load_timestamp,pr_to_po_ageing,pr_approval_ageing,po_approval_ageing,sla_breach_flag,pr_cycle_sla_breach_flag,po_cycle_sla_breach_flag,record_type,gold_load_date,gold_load_timestamp
2000126351,00480,ZNPR,2023-08-14,Tobi Emmaunel,Approved,2023-08-15,B-PO created,9900,ABC logistics Limited,9900,ABC logistics Limited - LAGOS,2,PAK,null,null,null,null,2025-11-24,2025-11-24T19:24:18.246Z,4500229324,00480,2000126351,24378,null,ZLPO,Local PO,2023-10-10,null,9900,ABC logistics Limited Porg,9900,ABC Logistics Limited,215,Tobi Emmaunel,1000019459,Maktabi Alhadeth Limited Co,Group A,Domestic Vendors/Sub-Contracto,External,GH,International,9900,ABC Logistics Limites - LAGOS,null,2023-10-10,Kazem Moustafa Badr Mohamed,2023,K,Cost center,ABC - LAG Division,00000001,05,05-Approved,2023-11-15,2023-10-19,2023-10-22,null,4000000040,"SIGN PEN, PILOT V-SIGN, BLACK, 12 PCS/PA",G121130,Stationery Expenses,NLAG,Non-Stock Materials,2,PAK,62.72,null,null,2025-11-24,2025-11-24T19:24:13.818Z,41,2,8,YES,NO,YES,PR_PO_MATCHED,2025-11-24,2025-11-24T19:32:18.832Z
2000130460,00050,ZNPR,2023-09-09,Christian PAT,Approved,2023-09-14,B-PO created,9900,ABC logistics Limited,9900,ABC logistics Limited - LAGOS,50,PAK,null,null,null,null,2025-11-24,2025-11-24T19:24:18.246Z,4500230817,00050,2000130460,24573,null,ZLPO,Local PO,2023-10-17,null,9900,ABC logistics Limited Porg,9900,ABC Logistics Limited,332,Christian PAT,1000019762,Al Falah Advanced Trading,Group A,Domestic Vendors/Sub-Contracto,External,GH,International,9900,ABC Logistics Limites - LAGOS,1002,2023-10-17,Javed Ahmed Khan,2023,null,null,null,00000000,05,05-Approved,2023-10-25,2023-10-18,2023-10-21,null,100000075426,"""BLADE, JIGSAW, NO. B 16 """"MAKITA"""" A-85684""",M031500,Accessories & Consumables,ZCON,Consumables,50,PAK,710.5,null,null,2025-11-24,2025-11-24T19:24:13.818Z,24,5,2,YES,YES,NO,PR_PO_MATCHED,2025-11-24,2025-11-24T19:32:18.832Z
2000126540,00010,ZNPR,2023-08-15,Christian PAT,Approved,2023-08-19,B-PO created,9900,ABC logistics Limited,9900,ABC logistics Limited - LAGOS,20,PAA,null,null,null,null,2025-11-24,2025-11-24T19:24:18.246Z,4500223373,00010,2000126540,23380,null,ZLPO,Local PO,2023-09-16,null,9900,ABC logistics Limited Porg,9900,ABC Logistics Limited,332,Christian PAT,1000019613,Yanbu Safety and Security,Group A,Domestic Vendors/Sub-Contracto,External,GH,International,9900,ABC Logistics Limites - LAGOS,1002,2023-09-16,Javed Ahmed Khan,2023,null,null,null,00000000,05,05-Approved,2023-09-01,2023-09-18,2023-09-23,null,100000074035,"SAFETY SHOES, LABOUR, SIZE: 40",G131160,Safety Shoes,ZCON,Consumables,20,PAA,764.4,null,null,2025-11-24,2025-11-24T19:24:13.818Z,20,3,2,YES,YES,NO,PR_PO_MATCHED,2025-11-24,2025-11-24T19:32:18.832Z
2000126351,00180,ZNP

Record Count: 681
Column Count: 80
root
 |-- pr_purchaserequisition: string (nullable = true)
 |-- pr_itemnumber: string (nullable = true)
 |-- pr_documenttype: string (nullable = true)
 |-- pr_creationdate: date (nullable = true)
 |-- pr_createdby_name: string (nullable = true)
 |-- pr_approvalstatus: string (nullable = true)
 |-- pr_approveddate: date (nullable = true)
 |-- pr_processingstatus: string (nullable = true)
 |-- pr_companycode: string (nullable = true)
 |-- pr_companycodedesc: string (nullable = true)
 |-- pr_plant: string (nullable = true)
 |-- pr_plantdesc: string (nullable = true)
 |-- pr_orderqty: string (nullable = true)
 |-- pr_unitofmeasure: string (nullable = true)
 |-- pr_closedindicator: string (nullable = true)
 |-- pr_deletionindicator: string (nullable = true)
 |-- pr_recordskipindicator: string (nullable = true)
 |-- pr_lastchangedatetime: string (nullable = true)
 |-- pr_silver_load_date: date (nullable = true)
 |-- pr_silver_load_timestamp: timestamp (null

pr_purchaserequisition,pr_itemnumber,pr_documenttype,pr_creationdate,pr_createdby_name,pr_approvalstatus,pr_approveddate,pr_processingstatus,pr_companycode,pr_companycodedesc,pr_plant,pr_plantdesc,pr_orderqty,pr_unitofmeasure,pr_closedindicator,pr_deletionindicator,pr_recordskipindicator,pr_lastchangedatetime,pr_silver_load_date,pr_silver_load_timestamp,purchaseorder,po_itemnumber,po_purchaserequisition,po_serialnumber,po_returnsitem,po_purchasingdoctype,po_purchasingdoctypedesc,po_purchasingdocdate,po_deletionindicator,po_purchasingorg,po_purchasingorgdesc,po_companycode,po_companycodedesc,po_purchasinggroup,po_purchasinggroupdesc,po_vendornumber,po_vendorname,po_vendoraccountgroup,po_vendoraccountgroupdesc,po_supplierclassification,po_countrykey,po_localorinternational,po_plant,po_plantdesc,po_storageloc,po_createdon,po_createdby_name,po_documentyear,po_accountassignmentcat,po_accountassignmentcatdesc,po_accntassignmentcatvaluedesc,po_version,po_processingstatuscode,po_processingstatus,po_postingdate,po_approvaldate,po_deliverydate,po_paymentdate,material,materialdesc,materialgroup,materialgroupdesc,materialtype,materialtypedesc,po_orderquantity,po_unitofmeasure,po_netamount,po_recordskipindicator,po_lastchangedatetime,po_silver_load_date,po_silver_load_timestamp,pr_to_po_ageing,pr_approval_ageing,po_approval_ageing,sla_breach_flag,pr_cycle_sla_breach_flag,po_cycle_sla_breach_flag,record_type,gold_load_date,gold_load_timestamp
81,81,115,115,81,124,126,115,115,81,115,81,81,115,681,681,681,646,81,81,336,336,399,336,681,336,336,336,681,336,336,336,336,336,336,336,336,336,336,336,336,336,336,336,591,336,336,336,418,418,418,370,336,336,346,348,336,681,338,336,336,336,338,338,336,336,336,681,681,336,336,421,126,348,421,126,348,63,0,0


pr_purchaserequisition,pr_itemnumber,pr_documenttype,pr_creationdate,pr_createdby_name,pr_approvalstatus,pr_approveddate,pr_processingstatus,pr_companycode,pr_companycodedesc,pr_plant,pr_plantdesc,pr_orderqty,pr_unitofmeasure,pr_closedindicator,pr_deletionindicator,pr_recordskipindicator,pr_lastchangedatetime,pr_silver_load_date,pr_silver_load_timestamp,purchaseorder,po_itemnumber,po_purchaserequisition,po_serialnumber,po_returnsitem,po_purchasingdoctype,po_purchasingdoctypedesc,po_purchasingdocdate,po_deletionindicator,po_purchasingorg,po_purchasingorgdesc,po_companycode,po_companycodedesc,po_purchasinggroup,po_purchasinggroupdesc,po_vendornumber,po_vendorname,po_vendoraccountgroup,po_vendoraccountgroupdesc,po_supplierclassification,po_countrykey,po_localorinternational,po_plant,po_plantdesc,po_storageloc,po_createdon,po_createdby_name,po_documentyear,po_accountassignmentcat,po_accountassignmentcatdesc,po_accntassignmentcatvaluedesc,po_version,po_processingstatuscode,po_processingstatus,po_postingdate,po_approvaldate,po_deliverydate,po_paymentdate,material,materialdesc,materialgroup,materialgroupdesc,materialtype,materialtypedesc,po_orderquantity,po_unitofmeasure,po_netamount,po_recordskipindicator,po_lastchangedatetime,po_silver_load_date,po_silver_load_timestamp,pr_to_po_ageing,pr_approval_ageing,po_approval_ageing,sla_breach_flag,pr_cycle_sla_breach_flag,po_cycle_sla_breach_flag,record_type,gold_load_date,gold_load_timestamp
11.894273127753303,11.894273127753303,16.886930983847286,16.886930983847286,11.894273127753303,18.208516886930983,18.502202643171806,16.886930983847286,16.886930983847286,11.894273127753303,16.886930983847286,11.894273127753303,11.894273127753303,16.886930983847286,100.0,100.0,100.0,94.86049926578562,11.894273127753303,11.894273127753303,49.33920704845815,49.33920704845815,58.590308370044056,49.33920704845815,100.0,49.33920704845815,49.33920704845815,49.33920704845815,100.0,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,49.33920704845815,86.78414096916299,49.33920704845815,49.33920704845815,49.33920704845815,61.38032305433187,61.38032305433187,61.38032305433187,54.33186490455213,49.33920704845815,49.33920704845815,50.80763582966226,51.10132158590308,49.33920704845815,100.0,49.632892804698976,49.33920704845815,49.33920704845815,49.33920704845815,49.632892804698976,49.632892804698976,49.33920704845815,49.33920704845815,49.33920704845815,100.0,100.0,49.33920704845815,49.33920704845815,61.820851688693104,18.502202643171806,51.10132158590308,61.820851688693104,18.502202643171806,51.10132158590308,9.251101321585903,0.0,0.0


Distribution for: pr_approval_ageing


...

Checking outliers for: pr_to_po_ageing


pr_to_po_ageing
41
24
20
41
1
41
24
41
1
41


pr_companycode,avg_delay
9900,29.66923076923077
null,null


po_countrykey,avg_delay
GH,29.66923076923077
NG,null
null,null


EDA Complete.
